In [1]:
import pandas as pd
import networkx as nx

### Loading data

In [31]:
df_lbsdot = pd.read_excel(r'../data/tot_nt_df.xlsx','original_data')
df_lbsdot_18Q4 = df_lbsdot[df_lbsdot.date == 20181231]
cc = set(df_lbsdot_18Q4[['frcc','tocc']].stack())

### Generating Graph

In [69]:
g_trade = nx.DiGraph()
g_lbs = nx.DiGraph()

In [33]:
# A way to add edge
# for x in df_lbsdot_18Q4.index:
#     g_trade.add_edge(df_lbsdot_18Q4.loc[x,'frcc'],df_lbsdot_18Q4.loc[x,'tocc'],weight=df_lbsdot_18Q4.loc[x,'trdval'])
#     g_lbs.add _edge(df_lbsdot_18Q4.loc[x,'frcc'],df_lbsdot_18Q4.loc[x,'tocc'],weight=df_lbsdot_18Q4.loc[x,'finval'])    

In [70]:
g_lbs.add_weighted_edges_from(list(df_lbsdot_18Q4.apply(lambda x: (x['frcc'], x['tocc'], x['finval']),axis=1)))

df_lbs_deg = pd.DataFrame([x for x in g_lbs.degree(weight='weight')], columns = ['cc','degree'])
df_lbs_ind = pd.DataFrame([x for x in g_lbs.in_degree(weight='weight')], columns=['cc','indegree'])
df_lbs_outd = pd.DataFrame([x for x in g_lbs.out_degree(weight='weight')], columns=['cc','outdegree'])

df_lbs_degree = pd.merge(df_lbs_deg,df_lbs_ind,on='cc')
df_lbs_degree = pd.merge(df_lbs_degree, df_lbs_outd, on='cc')

In [71]:
g_trade.add_weighted_edges_from(list(df_lbsdot_18Q4.apply(lambda x: (x['frcc'], x['tocc'], x['trdval']),axis=1)))

df_tde_deg = pd.DataFrame([x for x in g_trade.degree(weight='weight')], columns = ['cc','degree'])
df_tde_ind = pd.DataFrame([x for x in g_trade.in_degree(weight='weight')], columns=['cc','indegree'])
df_tde_outd = pd.DataFrame([x for x in g_trade.out_degree(weight='weight')], columns=['cc','outdegree'])

df_tde_degree = pd.merge(df_tde_deg,df_tde_ind,on='cc')
df_tde_degree = pd.merge(df_tde_degree, df_tde_outd, on='cc')

In [72]:
df_tde_degree.head()

,cc,degree,indegree,outdegree
0,FR,-3701912480,21856594986,-25558507466
1,EE,153552991,153552991,0
2,IE,-12767967689,-23814061788,11046094099
3,SK,-2628003273,-2628003273,0
4,TR,-632680739,-2148358539,1515677800


### Add country information

In [44]:
from pymongo import MongoClient

In [45]:
mongoclient = MongoClient(unicode_decode_error_handler = 'ignore')

In [61]:
df_country = pd.DataFrame(list(mongoclient.lbsdot.countrytable.find({},{"_id":0,"iso2":1,"iso3":1,"isoname":1, "lat":1,"lng":1})))

In [62]:
df_country.head(1)

,iso2,iso3,isoname,lat,lng
0,AF,AFG,Afghanistan,33.93911,67.709953


In [74]:
df_lbs_degree = pd.merge(df_lbs_degree, df_country, left_on='cc', right_on='iso2')
df_tde_degree = pd.merge(df_tde_degree, df_country, left_on='cc', right_on='iso2')

### Save degree result to Excel

In [76]:
xls_writer = pd.ExcelWriter(r'../data/degreeresult.xlsx')
df_lbs_degree.to_excel(xls_writer,'lbs',index=False)
df_tde_degree.to_excel(xls_writer,'dot',index=False)
xls_writer.close()

In [75]:
df_lbs_degree.head()

,cc,degree,indegree,outdegree,iso2,iso3,isoname,lat,lng
0,FR,285794.659,457363.401,-171568.742,FR,FRA,France,46.227638,2.213749
1,EE,4660.944,4660.944,0.000,EE,EST,Estonia,58.595272,25.013607
2,IE,70150.295,42348.972,27801.323,IE,IRL,Ireland,53.412910,-8.243890
3,SK,19176.371,19176.371,0.000,SK,SVK,Slovakia,48.669026,19.699024
4,TR,41692.386,74078.115,-32385.729,TR,TUR,Turkey,38.963745,35.243322


In [79]:
from urllib.request import urlopen as uReq
from urllib.request import Request, urlopen
headers={
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
}

req = Request(r'https://icobench.com/icos?', data=None, headers=headers)

In [80]:
webpage = urlopen(req).read()

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
webpage